In [1]:
import os

In [2]:
%pwd

'c:\\Edu\\Python\\MLOPS_projects\\mlops_nlp_summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Edu\\Python\\MLOPS_projects\\mlops_nlp_summarizer'

In [47]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [48]:
from mlops_NLP_Text_Summarization.constants import *
from mlops_NLP_Text_Summarization.utils.common import read_yaml, create_directories

In [49]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_training_config

In [50]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [51]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            num_train_epochs=self.config.num_train_epochs, 
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, 
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay=self.config.weight_decay, 
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, 
            eval_steps=self.config.eval_steps, 
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
         ) 


        #trainer_args = TrainingArguments(    < - hard coded values
        #    output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
        #    per_device_train_batch_size=1, per_device_eval_batch_size=1,
        #    weight_decay=0.01, logging_steps=10,
        #    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
        #    gradient_accumulation_steps=16
        #) 

        trainer = Trainer(model=model_pegasus, 
                          args=trainer_args,
                          tokenizer=tokenizer, 
                          data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["train"], 
                          eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [52]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_trainer_config = ModelTraining(config=model_training_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-08-07 00:03:24,033: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-07 00:03:24,036: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-07 00:03:24,037: INFO: common: created directory at: artifacts]
[2023-08-07 00:03:24,038: INFO: common: created directory at: artifacts/model_training]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [38]:
print(torch.version.cuda)

None
